# MPG

We've looked at MPG before. It's a small dataset, but has some really nice categoricals.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


mpg = pd.read_csv('../files/mpg.csv', index_col=0)
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


The key to finding categoricals is doing a nunique on the columns. Columns with low value counts may very well be a categorical.

In [2]:
mpg.nunique()

manufacturer    15
model           38
displ           35
year             2
cyl              4
trans           10
drv              3
cty             21
hwy             27
fl               5
class            7
dtype: int64

* Year only has 2 values, but i still a continuos field (although it does show that this isn't the greatest dataset around).
* Cyl has 4 values, the amount of cylinders. This is a good order categorical.
* Trans has 10 different values, but another intriguing thing going on: "manual(m5)" it combines 2 pieces of data: Manual/automatic and the number of gears.
* drv is an unordered categorical (unless you feel like a front-wheel drive is better than a 4x4?)
* fl is fuel type, unordered categorical
* class is also an unordered categorical

(When doing this kind of analysis, always have small code-block on hand where you can quickly check the different values in the column, like the one just below.)

In [3]:
mpg["fl"].unique()

array(['p', 'r', 'e', 'd', 'c'], dtype=object)

## The metric system

Who knows what a mile/gallon is? A model won't care, it's just a scaled number, but we do.

Create two new columns, containing "clkm" (city litres per km) from "cty", city miles per gallon and "hwlkm" (high way...) from "hwy".

In [4]:
#DELETE
mpg['clkm'] = [ (100 * 3.785411784)/(1.609344 * value) for value in mpg['cty']]
mpg['hwlkm'] = [ (100 * 3.785411784)/(1.609344 * value) for value in mpg['hwy']]

mpg.head(10)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,clkm,hwlkm
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,13.067477,8.110848
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,11.200694,8.110848
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11.760729,7.587567
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,11.200694,7.840486
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,14.700911,9.046715
6,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact,13.067477,9.046715
7,audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact,13.067477,8.711651
8,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact,13.067477,9.046715
9,audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact,14.700911,9.408583
10,audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact,11.760729,8.400521


## Ordered categoricals

Let's begin with the ordered categoricals. When doing graphs we'd do what we did in the previous Diamonds-exercise:

```Python
cut_type = CategoricalDtype(categories=[...
```

But this is a model-building course, not a graph-building course. So we want to translate these values into numerical values. And it's a pity, but for the number of cylinders, this is done. The values in the column are 4, 5, 6 and 8, which is an integer value that increases as the number of cylinders gets bigger. Any model will take that into account.

But lets, for arguments sake, say there is an order in fuel types. We'll stick to the following column:

| Code  | Fuel Type                    | Environmental Impact (CO₂, NOx, etc.)                                      |
| ----- | ---------------------------- | -------------------------------------------------------------------------- |
| `'r'` | Regular unleaded petrol      | High emissions                                                             |
| `'p'` | Petrol                       | High emissions                                                             |
| `'d'` | Diesel                       | Lower CO₂ than petrol, but higher NOx/particulates — still high overall    |
| `'c'` | CNG (compressed natural gas) | Cleaner than petrol/diesel, but still fossil fuel                          |
| `'e'` | Electric                     | Lowest emissions (assuming clean grid)                                     |

How would we encode this into a model? First you have to choose which gets a lower number: bad for the environment or low emissions? It matters for us as humans, but for a model it doesn't. If there were a column about the taxes for every vehicle and these were to go up for cars as they get worse for the environment, then 0 = bad and 4 = good will yield a negative correlation. 0 = good and 4 = bad would yield a positive correlation. In any case the model will have the information needed to predict one based on the other.

So encode this field so every fuel type gets a number in the order stated in the table above.

In [5]:
#DELETE
fuel_map = {'e': 0, 'c': 1, 'd': 2, 'p': 3, 'r': 4}
mpg['fuel_encoded'] = mpg['fl'].map(fuel_map)
mpg.head()


,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,clkm,hwlkm,fuel_encoded
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,13.067477,8.110848,3
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,11.200694,8.110848,3
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11.760729,7.587567,3
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,11.200694,7.840486,3
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,14.700911,9.046715,3


## Unordered categoricals

Unordered categoricals (like 'red', 'blue', 'green' or 'brand A', 'brand B') should not be left as raw text unless you're using a model that explicitly supports it (like CatBoost).

The best way to encode them depends on the model you're using and the cardinality (number of unique categories).

Let's try one-hot encoding the 'drv'-column.


In [6]:
#DELETE
drv_encoded = pd.get_dummies(mpg['drv'], prefix='drv')
mpg = pd.concat([mpg, drv_encoded], axis=1)
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,clkm,hwlkm,fuel_encoded,drv_4,drv_f,drv_r
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,13.067477,8.110848,3,False,True,False
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,11.200694,8.110848,3,False,True,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11.760729,7.587567,3,False,True,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,11.200694,7.840486,3,False,True,False
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,14.700911,9.046715,3,False,True,False


Pros:
* Interpretable
* No ordinal assumptions

Cons:
* Sparse/high-dimensional if lots of categories
* Can hurt tree model performance if cardinality is high

Try a label-encoder for class next.

In [7]:
#DELETE
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
mpg['class_encoded'] = label_encoder.fit_transform(mpg[col])
mpg.head()

NameError: name 'col' is not defined

Pros:
* Simple, compact
* Trees treat integers as categories, not as ordered numbers

Cons:
* Not safe for linear models — they’ll assume order where none exists

## Feature engineering

Finally, trans, the dual-valued column. First split it into two different columns.

In [ ]:
#DELETE
mpg[['trans_type', 'trans_detail']] = mpg['trans'].str.extract(r'([a-z]+)\(([^)]+)\)')
mpg.head()

Which leaves us with two more categorical columns that we can apply all of the previous rules to.

And what is next?

- Encode trans_type and trans_detail with one-hot or a label-encoder
- Maybe encode manufacturer, but not with one-hot encoding (has 15 different values)
- Drop all non-digit-columns (booleans are fine too)
- Train a a model and start predicting!

You're model should improve, but inference will become more difficult. (Is audi 1 or 2? And which number does 'CNG' have?)